Fitness Fatigue Models Illustrative Code
=========================================

This is a companion notebook illustrating the fitness fatigue models discussed in our review series. It is a Kaggle Notebook with dependencies on data and scripts hosted on Kaggle.com. To run or edit this notebook, please visit the latest [Kaggle version](https://www.kaggle.com/baogorek/fitness-fatigue-models-illustrative-code) (if you are not already on Kaggle.com!).

## Dependencies
 - [example_loads.csv](https://www.kaggle.com/baogorek/example-training-loads), a data set of example training loads created by @bsh2020.
 - [ffmfunctions.R](https://www.kaggle.com/baogorek/ffmfunctions), an R script containing functions relevant to the Fitness Fatigue model and variations discussed in our review papers
 

In [ ]:
# These paths are relative to Kaggle servers
source("../usr/lib/ffmfunctions/ffmfunctions.R")
example_loads <- read.csv('../input/example-training-loads/example_loads.csv')

set.seed(523445)  # Seed for reproducible output

## Training loads

In this research, the raw training loads $\omega^i$ are taken to be exogenous, i.e., coming from some predetermined plan, not a downstream impact of actual performance. It is represented in the code as `w`.

As noted in the dependencies, this notebook is accompied by an extensive training load dataset, which uses real exercises and separates upper body from lower body training impulses. These may be used below by choosing `training_type` to be either `"upper body"` or `"lower body"`. In addition there is also a fully synthetic option, `"synthetic"` that creates variation oportunistically, such as a long rest in the middle of the program so that fitness has time to fade. 


In [ ]:
# The Training Plan -----------------------------------------------------------

# "upper body", "lower body", "synthetic"
training_type <- "upper body" 

if (training_type == "synthetic") {
  w <- rep(c(seq(10, 50), rep(20, 14)), 5)
  w <- c(w, rep(0, 100), w)  #  Adding long rest!
} else if (training_type == "upper body") {
  w <- as.numeric(example_loads$tl_upper_fitness)
} else if (training_type == "lower body") {
  w <- as.numeric(example_loads$tl_lower_fitness)
}

plot(w, main = "Training impulses for this demonstration", xlab = "time")

## The Basic Model

The basic 5-parameter Fitness Fatigue Model from our review is:

$$
\text{E}(p_n | \omega_1, \ldots, \omega_{n-1}) = p^{\star} + k_g \sum_{i=1}^{n-1} \omega_i \cdot e^{-\frac{(n-i)}{\tau_g}}
 - k_h \sum_{i=1}^{n-1} \omega_i \cdot e^{-\frac{(n-i)}{\tau_h}}.
$$
Assuming Gaussian error, there is a 6th parameter, $\sigma$, as
$$
\epsilon_n \text{ are i.i.d. } N(0, \sigma^2).
$$

In this section's code blocks, we will instantiate it, simulate from it, predict with it, and estimate unknown parameters using both gradient descent (with analytical gradients) and the L-BFGS-B algorithm.

In [ ]:
# Basic FFM -------------------------------------------------------------------
ffm_basic <- create_ffm_model(p_star = 400, k_g = 1, k_h = 3, tau_g = 60,
                              tau_h = 15, sigma = 20)

print(ffm_basic)

In [ ]:
df <- simulate(ffm_basic, w)

# Predictions with true parameters
pred_true_df <- make_predictions(ffm_basic, w)
plot(df$y, main = "Simulated data with FFM predictions (true parameters)")
points(pred_true_df$y_hat, col = 'blue')

### Estimation in the basic model
Below we approach estimation of the unknown parameters of the basic Fitness Fatigue Model. As this is a non-convex estimation problem, good starting values are important. In our review, we describe an approach to data-driven starting values. This is implemented below. Note that the initialization results in a valid model in itself.

For the basic model, the inputs to the initialization are the data set and a course grid of potential time constants. These have default values but are specified below for transparency.

In [ ]:
# Estimating basic model, first get starting values from data set
ffm_from_data <- initialize_ffm_from_data(df, tau_g_seq = c(10, 50, 90),
                                          tau_h_seq = c(5, 10, 20))
print(ffm_from_data)

#### Maximum Likelihood Estimation
Maximum Likelihood Estimation is performed with R's base `optim` function using the L-BFGS-B algorithm. The method of choosing starting values and upper and lower bounds is described in the review.

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df)
print(ffm_ml)

#### Gradient Descent
The gradient vector of the sum of squared residuals with respect to the fitness fatigue model is analytically tractible and may be used in a gradient descent algorithm. This method is included for comparison with the L-BFGS-G method used in R's base `optim` function. While it is possible to reduce the error dramatically from an intial condition, scaling is important.

This procedure normalizes the gradient, then applies individual scaling of the parameters before applying the usual $\lambda$ tuning rate parameter (See [jermwatt.github.io](https://jermwatt.github.io/machine_learning_refined/notes/3_First_order_methods/3_9_Normalized.html) for an explanation.) It may be necessary to run the algorithm multiple times with different scalings and values of $\lambda$. The method is thus

$$
\theta_k^{\star} = \theta_{k - 1}^{\star} - \lambda \frac{\nabla g(\theta_{k - 1})}{\Vert \nabla g(\theta_{k - 1}) \Vert_2 }
$$

where $\theta$ are the 5 unknown model expectation parameters (i.e., not including error variance), $\theta_k^{\star}$ is $\theta$ after elementwise division with a parameter scaling vector (`parscale` below), and $g(\cdot)$ is the residual mean squared error function.

For demonstration purposes, the starting values will be strategically far away from their true values. As the user will see, it's challenging to match the performance of L-BFGS-B, and multiple rescalings are necessary. Note the parameters that move and the ones that don't

In [ ]:
# Demonstration - to get feel for Gradient Descent

ffm_close <- create_ffm_model(p_star = 385, k_g = .5, k_h = 2.5, tau_g = 52,
                              tau_h = 12, sigma = 15)

# Initial try - note that p_star isn't moving much
ffm_gd <- increase_likelihood_by_gradient(ffm_close, df, reps = 1000,
                                          lambda = .001, thin = 100)

# Adjust parscale so that p_star moves
ffm_gd <- increase_likelihood_by_gradient(ffm_gd, df, reps = 2000,
                                          lambda = .001, thin = 200,
                                          parscale = c(.001, 4, 1, .05, .25))
print(ffm_gd)

## Incorporating initial values

To the basic 5-parameter Fitness Fatigue Model, we now relax the assumption of zero initial fitness and fatigue effects and add the parameters $q_g$ and $q_h$. These have the interpretation of fitness and fatigue present at time $i=0$ with a zero training impulse at time $i=0$.

$$
\begin{align}
\text{E}(p_n | \omega_1, \ldots, \omega_{n-1}) = p^{\star} &+ k_g \sum_{i=1}^{n-1} \omega_i \cdot e^{-\frac{(n-i)}{\tau_g}}
 - k_h \sum_{i=1}^{n-1} \omega_i \cdot e^{-\frac{(n-i)}{\tau_h}} \\
 &+ q_g \cdot e^ {-\frac{n}{\tau_g}} + q_h \cdot e ^ {-\frac{n}{\tau_h}}.
\end{align}
$$


In [ ]:
ffm_add_initial <- create_ffm_model(p_star = 400, k_g = 1, k_h = 3, tau_g = 60,
                                    tau_h = 15, sigma = 20, q_g = 500, q_h = 250)
print(ffm_add_initial)

#### Visual Confirmation of non-zero offsets
In the data frame head and plots below, confirm that fitness and fatigue have non-zero offsets.

In [ ]:
df <- simulate(ffm_add_initial, w)
head(df, 5)  # Look for fitness and fatigue to be around their starting values

In [ ]:
# See the initial values at work
plot(df$fitness, main = "fitness", xlab = "time")
plot(df$fatigue, main = "fatigue", xlab = "time")

In [ ]:
ffm_from_data <- initialize_ffm_from_data(df, estimate_initial = TRUE)

In [ ]:
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE)
print(ffm_ml)

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "predictions vs data for initial values model")
points(pred_df$y_hat, col = 'red')

## The Variable Dose-Response (VDR) Model

We now relax the assumption that the daily increase in fatigue depends only on the previous workout. In the variable dose-response (VDR) model, the daily increase to fatigue may depends on past training loads in an exponentially decaying manner.
$$
\begin{align}
\text{E}(p_n | \omega_1, \ldots, \omega_{n-1}) = p^{\star} &+ k_g \sum_{i=1}^{n-1} \omega_i \cdot e^{-\frac{(n-i)}{\tau_g}}
 - k_h \sum_{i=1}^{n-1} k_{h_2} ^ i \cdot e^{-\frac{(n-i)}{\tau_h}} \\
 &+ q_g \cdot e^ {-\frac{n}{\tau_g}} + q_h \cdot e ^ {-\frac{n}{\tau_h}},
\end{align}
$$

where
$$
k_{h_2} ^ i = \sum_{j=1}^i \omega_j \cdot e^{-\frac{(i-j)}{\tau_{h_2}}}.
$$

Below, notice how we brough down the value of $k_h$ from 3 to 1.5. If you set it at 3, you'll be in for some massive swings as fatigue now accumulates over multiple days.

In [ ]:
ffm_vdr <- create_ffm_model(p_star = 400, k_g = 1, k_h = 1.5, tau_g = 60,
                            tau_h = 15, sigma = 20,
                            tau_h2 = 3, 
                            q_g = 300, q_h = 250)
print(ffm_vdr)

In [ ]:
df <- simulate(ffm_vdr, w)
head(df, 5)

In [ ]:
# Fatigue can get pretty large with tau_h and tau_h2
plot(df$fitness, main = "Fitness with VDR")
plot(df$fatigue, main = "Fatigue with VDR")

In [ ]:
# Predictions with true parameters
pred_true_df <- make_predictions(ffm_vdr, w)
plot(df$y, main = "VDR simulated and predictions with true values")
points(pred_true_df$y_hat, col = 'blue')

#### Estimation of VDR parameters
Note that we include a time constant sequence for the new $\tau_{h_2}$ parameter in `initialize_ffm_from_data`.

In [ ]:
# Specify tau_h2_seq for data-driven VDR starting values
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          tau_h2_seq = c(1, 2, 5, 10, 15),
                                          estimate_initial = TRUE)

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE, tune_vdr = TRUE)
print(ffm_ml)

In [ ]:
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "predictions in VDR & initial based on ML Fit")
points(pred_df$y_hat, col = 'red')

## Using the Hill function
The Hill function is defined as

$$
Hill(\omega) = \kappa \left( \frac{\omega^\gamma}{\delta^\gamma + \omega^\gamma}\right)
$$
and, when used as a transformation to the raw training inputs, may mitigate the non-linearity in the training dose-response profile. Below is a demonstration of the Hill function applied to the same raw inputs as before, first with the basic model and then with all the other optional parameters.

The model is thus:

$$
\text{E}(p_n | \omega_1, \ldots, \omega_{n-1}) = p^{\star} + k_g \sum_{i=1}^{n-1} \omega_i^{\star} \cdot e^{-\frac{(n-i)}{\tau_g}}
 - k_h \sum_{i=1}^{n-1} \omega_i ^ {\star} \cdot e^{-\frac{(n-i)}{\tau_h}}
$$

where
$$
\omega_j^{\star} = \kappa \left( \frac{\omega_j^\gamma}{\delta^\gamma + \omega_j^\gamma}\right), j = 1, \ldots, n.
$$

In the following, $\kappa$ is not estimated but instead set to 100. This parameter is not estimable from data since $k_g$ and $k_h$ can "undo" the multiplier from $\kappa$. 

In [ ]:
ffm_hill <- create_ffm_model(p_star = 400, k_g = 1, k_h = 3, tau_g = 60,
                             tau_h = 15, sigma = 20,
                             gamma = 2, delta = 10, kappa = 100)
print(ffm_hill)

In [ ]:
w_hill <- get_hill_transformed_training(ffm_hill, w)
plot(w_hill ~ w, main = "Simulation-specified Hill Transformation")

In [ ]:
df <- simulate(ffm_hill, w)
head(df)

In [ ]:
# Predictions with true model
pred_df <- make_predictions(ffm_hill, w)
plot(df$y, main = "Predictions with True Model")
points(pred_df$y_hat, col = 'red')

In [ ]:
# Specify delta_seq and gamma_seq for data-driven hill starting values
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          delta_seq = c(.3, 1, 1.5, 5, 20),
                                          gamma_seq = c(.3, 1, 2, 5, 20))

In [ ]:
# One-shot maximum likelihood using L-BFGS-B
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_hill = TRUE)
print(ffm_ml)

In [ ]:
# Hill transformation analysis
w_hill_ml <- get_hill_transformed_training(ffm_ml, w)
plot(w_hill ~ w, 
     main = "Hill transformation - True (black) & Est (blue )")
points(w_hill_ml ~ w, col = 'blue')

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "Predictions with Hill-estimated mode")
points(pred_df$y_hat, col = 'red')

## "The Works" Model: VDR, Initial Values, and Hill transformation
Fitting a model based on two variables with ten parameters in the expectation is ambitious, but below we go for it. The model here is

$$
\begin{align}
\text{E}(p_n | \omega_1, \ldots, \omega_{n-1}) = p^{\star} &+ k_g \sum_{i=1}^{n-1} \omega_i^{\star} \cdot e^{-\frac{(n-i)}{\tau_g}}
 - k_h \sum_{i=1}^{n-1} k_{h_2} ^ i \cdot e^{-\frac{(n-i)}{\tau_h}} \\
 &+ q_g \cdot e^ {-\frac{n}{\tau_g}} + q_h \cdot e ^ {-\frac{n}{\tau_h}},
\end{align}
$$

where
$$
k_{h_2} ^ i = \sum_{j=1}^i \omega_j^{\star} \cdot e^{-\frac{(i-j)}{\tau_{h_2}}}
$$
and 
$$
\omega_j^{\star} = \kappa \left( \frac{\omega_j^\gamma}{\delta^\gamma + \omega_j^\gamma}\right), j = 1, \ldots, n.
$$

Again, $\kappa$ is not estimated but instead set to 100 for convenience.


In [ ]:
ffm_the_works <- create_ffm_model(p_star = 400, k_g = 1, k_h = .9, tau_g = 60,
                                  tau_h = 15, sigma = 20, tau_h2 = 3,
                                  gamma = 2, delta = 10, kappa = 100,
                                  q_g = 300, q_h = 250)
print(ffm_the_works)

In [ ]:
w_hill_the_works <- get_hill_transformed_training(ffm_the_works, w)
plot(w_hill_the_works ~ w, main = "Hill transformation")

In [ ]:
df <- simulate(ffm_the_works, w)
head(df)

In [ ]:
plot(df$fitness, main = "fitness", xlab = "time")
plot(df$fatigue, main = "fatigue", xlab = "time")

In [ ]:
pred_df <- make_predictions(ffm_the_works, w)
plot(df$y, main = "Simulated data and true model predictions")
points(pred_df$y_hat, col = 'red')

#### Estimation in "The Works" model
The initialization routine gets more complex, as now the grid must include both $\gamma$ and $\delta$ sequences. While the grid has become very large, as each iteration is a closed form regression the starting value-generating procedure still returns fairly quickly.

In [ ]:
ffm_from_data <- initialize_ffm_from_data(df,
                                          tau_g_seq = c(10, 50, 80),
                                          tau_h_seq = c(5, 10, 20),
                                          tau_h2_seq = c(1, 2, 5, 10),
                                          delta_seq = c(.3, 1, 1.5, 5, 20),
                                          gamma_seq = c(.3, 1, 2, 5, 20),
                                          estimate_initial = TRUE)

In [ ]:
ffm_ml <- maximize_likelihood(ffm_from_data, df, tune_initial = TRUE,
                              tune_vdr = TRUE, tune_hill = TRUE)
print(ffm_ml)

In [ ]:
# Predictions with estimated model
pred_df <- make_predictions(ffm_ml, w)
plot(df$y, main = "Predictions from ML-estimated 'The Works' model")
points(pred_df$y_hat, col = 'red')

In [ ]:
w_hill_ml <- get_hill_transformed_training(ffm_ml, w)
plot(w_hill ~ w, 
     main = "Hill transformation - True (black) & Est (blue )")
points(w_hill_ml ~ w, col = 'blue')